# Import Statements

In [31]:
from google.colab import drive
import nltk
import re
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
import os
import nltk

In [32]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# File Unzip

In [33]:
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [4]:
#('/content/MyDrive/MyDrive/IR/Humor,Hist,Media,Food.zip')
!unzip /content/MyDrive/MyDrive/IR/Humor,Hist,Media,Food.zip > /dev/null

In [34]:
dataset_path='/content/Humor,Hist,Media,Food'
alldirs = [dataset_path+'/'+d for d in os.listdir(os.path.join(dataset_path))]
alldirs.sort()
FileContents=[]
for path in alldirs:
  #print(path)
  file = open(path,'rb')
  FileContents.append(file.read()) 

# Pre processing

In [35]:
nltk_tokens=[]
for i in range(len(FileContents)):
  #Conversion to lowercase
  FileContents[i]=str(FileContents[i].lower()).replace("\\n"," ").replace("\\r"," ").replace("\\t"," ").replace("\n"," ").replace("\r"," ").replace("\t"," ").strip()
  #Remove Special Characters
  FileContents[i]=str(re.sub('[^A-Za-z0-9]+', ' ',FileContents[i]))
  #Word Tokenization
  nltk_tokens.append(nltk.word_tokenize(FileContents[i]))


#Lemmatization
lemmatizer = WordNetLemmatizer()
lemmatized_output=[]
for i in range(len(nltk_tokens)):
  lemmatized_output.append([lemmatizer.lemmatize(w) for w in nltk_tokens[i]])  
  
  
#Remove Stop Words
stop_words = set(stopwords.words('english'))
preprocessed=[]
for i in lemmatized_output:
	temp=[]
	for r in i:
		if not r in stop_words:
			temp.append(r)
	preprocessed.append(temp)

# Inverted Index

In [36]:
dict = {}
  
for i in range(len(preprocessed)):
    tokens=preprocessed[i]
    for item in tokens:
        if item not in dict:
            dict[item] = set()

        if item in dict:
            dict[item].add(i)

# Query

In [37]:
#Finding the set of docs in which a token is present
def finddocset(input):
  if input in dict:
    return dict[input]
  else:
    return set()

In [38]:
def OR(set1,set2):
  list1=list(set1)
  list2=list(set2)
  list1.sort()
  list2.sort()
  num_comp=0
  i=0
  j=0
  result=[]
  while i<len(list1) and j<len(list2):
    if list1[i]<list2[j]:
      result.append(list1[i])
      i+=1
    elif list1[i]>list2[j]:
      result.append(list2[j])
      j+=1
    else:
      result.append(list1[i])
      i+=1
      result.append(list2[j])
      j+=1
    num_comp+=1

  while i<len(list1):
    result.append(list1[i])
    i+=1
  
  while j<len(list2):
    result.append(list2[j])
    j+=1

  result=set(result)
  return (result,num_comp)


In [39]:
def AND(set1,set2):
  list1=list(set1)
  list2=list(set2)
  list1.sort()
  list2.sort()
  len1=len(list1)
  len2=len(list2)

  if len1>len2:
    temp=list1
    list1=list2
    list2=temp

  # Now list1 has less elements than lest2
  d={}
  for k in list2:
    d[k]=1
  
  result=[]
  i=0
  while i<len(list1):
    if list1[i] in d:
      result.append(list1[i])
    i+=1

  num_comp=len(list1)
  
  result=set(result)
  return (result,num_comp)


In [40]:
def ANDNOT(set1,set2):
  list1=list(set1)
  list2=list(set2)
  list1.sort()
  list2.sort()
  
  d={}
  for k in list2:
    d[k]=1
  
  result=[]
  i=0
  while i<len(list1):
    if list1[i] not in d:
      result.append(list1[i])
    i+=1

  num_comp=len(list1)
  
  result=set(result)
  return (result,num_comp)

In [24]:
#NOT
def NOT(set1):
  universe=set()
  for i in range(len(alldirs)):
    universe.add(i)

  return universe.difference(set1)

In [41]:
# QUERY PREPROCESSING

ip='lion stood thoughtfully for a moment'
#ip='telephone paved roads'
operators=['OR','OR','OR']
nltk_tokens=[]

#Conversion to lowercase
ip=str(ip.lower()).replace("\\n"," ").replace("\\r"," ").replace("\\t"," ").replace("\n"," ").replace("\r"," ").replace("\t"," ").strip()
#Remove Special Characters
ip=str(re.sub('[^A-Za-z0-9]+', ' ',ip))
#Word Tokenization
nltk_tokens.append(nltk.word_tokenize(ip))
#Lemmatization
lemmatizer = WordNetLemmatizer()
lemmatized_output=[]
for i in range(len(nltk_tokens)):
  lemmatized_output.append([lemmatizer.lemmatize(w) for w in nltk_tokens[i]]) 

  
  
#Remove Stop Words
stop_words = set(stopwords.words('english'))
tokens=[]
for i in lemmatized_output:
	temp=[]
	for r in i:
		if not r in stop_words:
			temp.append(r)
	tokens.append(temp)
 
tokens=tokens[0]

In [42]:
tokens

['lion', 'stood', 'thoughtfully', 'moment']

In [43]:
m=0
if len(tokens)!=len(operators)+1:
  print("Incorrect Number of operands")
else:
  result=finddocset(tokens[0])
  comp=0
  # Handle the case that a token is not present in the document
  while m<len(operators):
    if operators[m]=='OR':
      (result,c)=OR(result,finddocset(tokens[m+1]))
      comp+=c
    if operators[m]=='AND':
      (result,c)=AND(result,finddocset(tokens[m+1]))
      comp+=c
    if operators[m]=='AND NOT':
      (result,c)=ANDNOT(result,finddocset(tokens[m+1]))
      comp+=c
    if operators[m]=='OR NOT':
      '''(result,c)=AND(result,finddocset(tokens[m+1]))
      comp+=c'''
      temp=NOT(finddocset(tokens[m+1]))
      (result,c)=OR(result,temp)
      comp+=c
    m+=1
  

In [44]:
print(str(len(result))+" documents were retrieved")
print(str(comp)+" comparisons were made")
filenames=[]
for i in result:
  filenames.append(alldirs[i][31:])

211 documents were retrieved
400 comparisons were made


In [29]:
print("The list of documents are")
filenames.sort()

The list of documents are


In [30]:
filenames

['a_tv_t-p.com',
 'aeonint.txt',
 'allusion',
 'ambrose.bie',
 'anim_lif.txt',
 'anime.lif',
 'annoy.fascist',
 'art-fart.hum',
 'b-2.jok',
 'badday.hum',
 'barney.txt',
 'bbh_intv.txt',
 'beauty.tm',
 'beesherb.txt',
 'bitnet.txt',
 'bmdn01.txt',
 'boneles2.txt',
 'butwrong.hum',
 'bw-phwan.hat',
 'bw.txt',
 'cabbage.txt',
 'calculus.txt',
 'candy.txt',
 'cartoon.law',
 'cartoon.laws',
 'cartoon_.txt',
 'chickenheadbbs.txt',
 'childhoo.jok',
 'christop.int',
 'clancy.txt',
 'classicm.hum',
 'cmu.share',
 'cogdis.txt',
 'collected_quotes.txt',
 'commutin.jok',
 'computer.txt',
 'conan.txt',
 'consp.txt',
 'cookie.1',
 'coyote.txt',
 'cuchy.hum',
 'cybrtrsh.txt',
 'dead3.txt',
 'dead4.txt',
 'dead5.txt',
 'deep.txt',
 'devils.jok',
 'dingding.hum',
 'doggun.sto',
 'drinks.gui',
 'dromes.txt',
 'drunk.txt',
 'dthought.txt',
 'econridl.fun',
 'engineer.hum',
 'epi_.txt',
 'epi_tton.txt',
 'episimp2.txt',
 'epitaph',
 'eskimo.nel',
 'exam.50',
 'facedeth.txt',
 'fascist.txt',
 'female.jok'

In [ ]:
del input

In [ ]:
number_of_queries=input("Enter Number of Queries ")
number_of_queries=int(number_of_queries)

while number_of_queries>0:
  try:
    del ip
    del op
  except:
    pass
  number_of_queries-=1
  ip=input("Enter Query sentence ")
  op=input("Enter Operands ")
  op=op.split(',')
  op[0]=op[0][1:]
  op[-1]=op[-1][:-1]
  print(op)
  nltk_tokens=[]

  #Conversion to lowercase
  ip=str(ip.lower()).replace("\\n"," ").replace("\\r"," ").replace("\\t"," ").replace("\n"," ").replace("\r"," ").replace("\t"," ").strip()
  #Remove Special Characters
  ip=str(re.sub('[^A-Za-z0-9]+', ' ',ip))
  #Word Tokenization
  nltk_tokens.append(nltk.word_tokenize(ip))
  #Lemmatization
  lemmatizer = WordNetLemmatizer()
  lemmatized_output=[]
  for i in range(len(nltk_tokens)):
    lemmatized_output.append([lemmatizer.lemmatize(w) for w in nltk_tokens[i]]) 

    
    
  #Remove Stop Words
  stop_words = set(stopwords.words('english'))
  tokens=[]
  for i in lemmatized_output:
    temp=[]
    for r in i:
      if not r in stop_words:
        temp.append(r)
    tokens.append(temp)
  
  tokens=tokens[0]
  m=0
  if len(tokens)!=len(operators)+1:
    print("Incorrect Number of operands")
    continue
  else:
    result=finddocset(tokens[0])
    comp=0
    # Handle the case that a token is not present in the document
    while m<len(operators):
      if operators[m]=='OR':
        (result,c)=OR(result,finddocset(tokens[m+1]))
        comp+=c
      if operators[m]=='AND':
        (result,c)=AND(result,finddocset(tokens[m+1]))
        comp+=c
      if operators[m]=='AND NOT':
        (result,c)=ANDNOT(result,finddocset(tokens[m+1]))
        comp+=c
      if operators[m]=='OR NOT':
        
        temp=NOT(finddocset(tokens[m+1]))
        (result,c)=OR(result,temp)
        comp+=c
      m+=1

  print(str(len(result))+" documents were retrieved")
  print(str(comp)+" comparisons were made")
  filenames=[]
  for i in result:
    filenames.append(alldirs[i][31:])

  print("The list of documents are")
  print(filenames)
  del ip
  del op
    


Enter Number of Queries 1
Enter Query sentence lion stood thoughtfully for a moment
Enter Operands [OR,OR,OR]
['OR', 'OR', 'OR']
208 documents were retrieved
391 comparisons were made
The list of documents are
['initials.rid', 'a_tv_t-p.com', 'three.txt', 'throwawa.hum', 'insult.lst', 'insults1.txt', 'timetr.hum', 'tnd.1', 'top10.txt', 'top10st2.txt', 'is_story.txt', 'tpquotes.txt', 'ivan.hum', 'aeonint.txt', 'japantv.txt', 'jayjay.txt', 'jc-elvis.inf', 'allusion', 'ambrose.bie', 'ukunderg.txt', 'anim_lif.txt', 'anime.lif', 'annoy.fascist', 'valujet.txt', 'kaboom.hum', 'kanalx.txt', 'various.txt', 'vonthomp', 'wacky.ani', 'wagon.hum', 'art-fart.hum', 'wedding.hum', 'lawyer.jok', 'b-2.jok', 'lbinter.hum', 'whoops.hum', 'let.go', 'letgosh.txt', 'badday.hum', 'wimptest.txt', 'lif&love.hum', 'lifeimag.hum', 'lifeonledge.txt', 'barney.txt', 'lion.jok', 'lion.txt', 'lions.cat', 'worldend.hum', 'bbh_intv.txt', 'llong.hum', 'beauty.tm', 'xibovac.txt', 'lozerzon.hum', 'luggage.hum', 'beesherb.t